I took the hero dataset that I used for the previous assignment and add random weights so taht the island method could be applied. I downloaded and hosted the data on my github and call it in from the github URL

In [10]:

import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from scipy import stats
import numpy as np
import random

def trim_edges(g,weight=1):
    g2=nx.Graph()
    for f, to, edata in g.edges(data=True):
        if edata['weight'] > weight:
            g2.add_edge(f,to,edata)
    return g2

def tidy(g, weight):
    g_temp = nx.Graph()
    edge_bunch2 = [i for i in g.edges(data=True) if i[2]['weight'] > weight]    
    g_temp.add_edges_from(edge_bunch2)
    return g_temp

def island_method(g,iterations=5):
    weights = [edata['weight'] for f,to,edata in g.edges(data=True)]
    mn=int(min(weights))
    mx=int(max(weights))
    step=int((mx-mn)/iterations)
    return [[threshold, trim_edges(g,threshold)] for threshold in range(mn,mx,step)]

edge_data=pd.read_csv("https://raw.githubusercontent.com/aschwenker/620/master/hero_edges.csv")
node_data=pd.read_csv("https://raw.githubusercontent.com/aschwenker/620/master/hero_nodes.csv")
#edge_data = edge_data.iloc[:10]
#node_data = node_data.iloc[:10]
edge_tuples = [tuple(x) for x in edge_data.values]
node_tuples = [tuple(x) for x in node_data.values]
G = nx.Graph()

G.add_edges_from(edge_tuples)
G.add_nodes_from(node_tuples)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(0,10)
big_subg = [len(i) for i in nx.connected_component_subgraphs(G)]
big_subg = [len(i) for i in nx.connected_component_subgraphs(G) if len(i) > 1]

big_subg

[19029, 8, 11, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 4, 2, 2, 2, 3, 2, 2]

From this we can see that there are many small subgraphs and 1 larger one --  19029. Similar to the text book example this one large subgraph is still very large so we need to break it down further

Next I applied a function similar to the text book version to reduce the size of the network based on weight
after that I calculated the degree of centrality for the two types of nodes and compared them using the table.


In [20]:

island = tidy(G,1)


island_nodes = list(set(island))
island_node_data = node_data.loc[(node_data["node"].isin(island_nodes))]
island_node_data['degcen'] = 0
deg = nx.degree(island) 


for index, row in island_node_data.iterrows():
    island_node_data.loc[island_node_data['node'] == row['node'], 'degcen'] = deg[row['node']]


node_data_deg = island_node_data.groupby("type")['degcen'].mean().reset_index()


result = pd.concat([node_data_deg],axis = 1)
result = result.loc[:,~result.columns.duplicated()]
print(result)

C:\Users\aschwenker\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


    type     degcen
0  comic   6.371320
1   hero  12.714991


from this we can see that nodes for heros are much more central than comics in this subset of the data